In [1]:
from aqutils import scripts, cli
from pydent.planner import Planner
from pydent.browser import Browser
from mysession import nursery, production, benchapi
from tqdm import tqdm

cli = cli.entrypoint()
%matplotlib inline

/Users/justinvrana/Documents/GitHub/coral/coral/analysis/_sequencing/needle.py:11: UserWarning: NW alignment extension could not be imported, falling backon native Python version (~100 times slower).
  warnings.warn(message)


In [2]:
fvs = production.FieldValue.last(100)
for fv in fvs:
    fv.dump()

# Cache relationships and models
OperationType <> FieldType <> AllowableFieldType relationships

In [3]:
browser = Browser(production)

ots = browser.where({"deployed": True}, "OperationType")
browser.set_verbose(False)
production.set_verbose(False)
results = browser.recursive_retrieve(ots, {
    "field_types": {
        "allowable_field_types": {
            "object_type": [],
            "sample_type": [],
            "field_type": []
        }
    }
}
)

fts = results['field_types']
inputs = [ft for ft in fts if ft.role == 'input']
outputs = [ft for ft in fts if ft.role == 'output']

## Determine edges

In [4]:
def external_aft_hash(aft):
    if not aft.field_type:
        return str(uuid4())
    if aft.field_type.part:
        part = True
    else:
        part = False
    return "{object_type}-{sample_type}-{part}".format(
        object_type=aft.object_type_id,
        sample_type=aft.sample_type_id,
        part=part,
    )

def internal_aft_hash(aft):
    return "{operation_type}".format(
        operation_type=aft.field_type.parent_id,
        routing=aft.field_type.routing,
        sample_type=aft.sample_type_id
    )

fts = [ft for ft in results['field_types'] if ft.ftype == 'sample']
inputs = [ft for ft in fts if ft.role == 'input']
outputs = [ft for ft in fts if ft.role == 'output']

input_afts = []
for i in inputs:
    for aft in i.allowable_field_types:
        if aft not in input_afts:
            input_afts.append(aft)

output_afts = []
for o in outputs:
    for aft in o.allowable_field_types:
        if aft not in output_afts:
            output_afts.append(aft)
all_afts = input_afts + output_afts

# input_by_object_type_id = {}
# output_by_object_type_id = {aft.object_type_id: aft for aft in output_aft}
import itertools
production.set_verbose(True)

external_groups = {}
for aft in input_afts:
    external_groups.setdefault(external_aft_hash(aft), []).append(aft)
    
    
internal_groups = {}
for aft in output_afts:
    internal_groups.setdefault(internal_aft_hash(aft), []).append(aft)
    
    
# from tqdm import tqdm
matching = []
for oaft in tqdm(output_afts):
    hsh = external_aft_hash(oaft)
    externals = external_groups.get(hsh, [])
    for aft in externals:
        matching.append((oaft, aft))

for iaft in tqdm(input_afts):
    hsh = internal_aft_hash(iaft)
    internals = internal_groups.get(hsh, [])
    for aft in internals:
        matching.append((iaft, aft))

100%|██████████| 563/563 [00:00<00:00, 128273.84it/s]


## Edge weights

Here we use historical data for how plans are 'normally' connected and use that information to automatically plan experiments in the future.

Certainly, something like a *preference file* or a *specification file* how how thing can/should be connected would be exteremely useful.

In [5]:
from functools import reduce

plans = browser.last(300, "Plan")
browser.set_verbose(False)
browser.session.set_verbose(False)
browser.recursive_retrieve(plans, {"operations": {"field_values": ["allowable_field_type", "wires_as_source", "wires_as_dest"]}}, strict=False)
for p in plans:
    p.wires

all_wires = reduce((lambda x, y: x + y), [p.wires for p in plans])
all_operations = reduce((lambda x, y: x + y), [p.operations for p in plans])
browser.recursive_retrieve(all_wires[:], {
    "source": {
        "field_type": [],
        "operation": "operation_type",
        "allowable_field_type": []
    },
    "destination": {
        "field_type": [],
        "operation": "operation_type",
        "allowable_field_type": []
    }
}, strict=False)
pass

2018-11-09 14:45:00,866 - AqHTTP@http://52.27.43.242/ - INFO - BODY: {'arguments': {},
 'method': 'where',
 'model': 'Plan',
 'options': {'limit': '300', 'offset': '-1', 'reverse': 'True'}}


In [15]:
production.set_verbose(True)

from uuid import uuid4

def hash_afts(aft1, aft2):
    source_hash = external_aft_hash(aft1)
    dest_hash = external_aft_hash(aft2)
    return "{}->{}".format(source_hash, dest_hash)
    return str(uuid4())

def hash_wire(wire):
    func = external_aft_hash
    ifv = wire.source
    ofv = wire.destination
    if ifv and ofv:
        return hash_afts(ofv.allowable_field_type, ifv.allowable_field_type)
    return str(uuid4())

wire_hash_count = {}
wire_source_hash_count = {}

aft_pairs = []
for wire in all_wires:
    if wire.source and wire.destination:
        aft_pairs.append((wire.source.allowable_field_type, wire.destination.allowable_field_type))
for op in all_operations:
    for i in op.inputs:
        for o in op.outputs:
            aft_pairs.append((i.allowable_field_type, o.allowable_field_type))

for aft1, aft2 in tqdm(aft_pairs):
    if aft1 and aft2:
        aft_hash = hash_afts(aft1, aft2)
        wire_hash_count.setdefault(aft_hash, 0)
        wire_hash_count[aft_hash] += 1

        source_hash = external_aft_hash(aft1)
        wire_source_hash_count.setdefault(source_hash, 0)
        wire_source_hash_count[source_hash] += 1

def weight_edge(n1, n2):
    p = 1e-4
    t = 0
    if n1 and n1:
        n = wire_hash_count.get(hash_afts(n1, n2))*1.0
        t = wire_source_hash_count.get(external_aft_hash(n1))*1.0

    if t > 0:
        p = n/t
    w = (1-p)/(1+p)
    return 10/(1.0001 - w)


examples = []
for w in all_wires[:]:
    if w.source and w.destination:
        weight = weight_edge(w.source.allowable_field_type, w.destination.allowable_field_type)
        if weight > 0 and weight < 0.9:
            examples.append(w)

100%|██████████| 10834/10834 [00:00<00:00, 64862.38it/s]


In [16]:
for aft1, aft2 in matching[:10]:
    print(weight_edge(aft1, aft2))

15.018841051723879
15.018841051723879
15.018841051723879
14.99775033744938
14.99775033744938
14.99775033744938
14.99775033744938
14.99775033744938
14.99775033744938
14.99775033744938


## Filter graph

## Search graph

While there are many ways to connect Operations together that takes in a Fragment Stock and eventually produces a Yeast Glycerol Stock, many of these workflows do not make sense from a experimental standpoint. These *non-sensical* workflows are simply not what people do in the lab to produce a yeast glycerol stock.

As an example, it may be unconventional to connect a `Yeast Transformation` operation (which produces a yeast plate) directly to a `Yeast Overnight` operation (which produces a yeast overnight culture) since people typically check the plates first using `Chect Yeast Plate` before making a overnight culture. Even though the Transformation > Overnight path has fewer steps, this is far less common than the Transformation > Check Plate > Overnight path. During this search, we want to capture this concept.

To do this, we use historical data from past completed plans. In this example, say there 1000 `Yeast Transformations` in our historical data set with only 10 of these being connected directly to `Overnight` and the other 990 connected to `Check Plates`. We use these ratios (p=0.01 and p=0.99) to compute weights for the network of all possibly connected operations with less likely connections having a greater weight than other connections.

Example: Find the shortest path between a start and end nodes. Here, we show an example of the shortest path to take between a *Fragment Stock* and a *Yeast Glycerol Stock*.

Results: Given the OperationTypes in the Aquarium database, it would be most typical to product a plasmid from fragments in a gibson assembly. This plasmid would be miniprepped and transformed into yeast. Here we find this is the case. Without historical data, the search returns a much shorter and non-sensical workflow (Yeast Transformation to Define Culture Conditions, which, experimentally, is likely not our intention).

Note: The AutoPlanner computes probability of connecting different AllowableFieldTypes together, which accounts for external Operation to Operation connections as well as internal input to output connections within the same operation (it may be unconventional to use Fragment Stocks to produce another Fragment Stock in a Golden Gate Assembly, for example).

### Populating the graph

Aquarium plans are not experimentally valid unless all inputs in every operation has either an incoming wire or item, and all field values are populated with either a parameter or sample. We need to populate the graph with specific samples and items in order to autoplan an experiment.

To make a plan network, we do the following:

1. We add a START and GOAL node to the network.
2. We add edges from every node (AllowableFieldType) to its available items (location != deleted) that match its sample_type_id and object_type_id. **edge_weight=0**
3. Add edges from every item to the START node. **edge_weight=0**
4. We add undirected *sister edges* between relavant nodes. Sister edges occur between allowable_field_types that are attached to input FieldTypes within the same OperationType whose routing information is valid. Routing information between two afts are valid if either:
    * (a) the aft.field_type.routing is different between the two afts or 
    * (b) aft.sample_type_id between the two afts are the same.

**Goal**
Find the least costly valid plan.

**Definition of valid plan**
A plan is valid if (a) its network has no cycles, (b) the network has a single leaf node (including the *sister edges*)

#### Notes on Populating with items

It is far to intensive to query all items of a allowable_field_types. This should occur only after additional sample filtering has occured. This will mean establishing sample construction in cases where sample routing is switched along an edge. This can mean specifying how sample_types and operation_types interact?

# DONT FORGET TO INCLUDE PARTS/COLLECTIONS

### Dynamic programming

#### Construct template graph

A template graph represents **all** possible workflows. Each node is an AllowableFieldType. Connections are determined by:

1. internal connections where "input-{operation_type_id}" -> "output-{operation_type_id}"
2. external connections where "output-{object_type_id}-{sample_type_id}" -> "input-{object_type_id}-{sample_type_id}"

Weights are determined as described previously.

In [17]:
import networkx as nx

# new direction graph
G = nx.DiGraph()

# add weighted edges
for aft1, aft2 in matching:
    if aft1 and aft2:
        G.add_edge(aft1.id, aft2.id, weight=weight_edge(aft1, aft2))

print("Building Graph:")
print("  {} edges".format(len(matching)))
print("  {} nodes".format(len(G)))

all_afts = input_afts + output_afts

ignore_ots = browser.where({"category": "Control Blocks"}, "OperationType")
nodes = [aft.id for aft in all_afts if aft.field_type.parent_id and aft.field_type.parent_id not in [ot.id for ot in ignore_ots]]
subgraph = G.subgraph(nodes)
print("Graph size reduced from {} to {} nodes".format(len(G), len(subgraph)))
print("Example edges")
for e in list(subgraph.edges)[:3]:
    aft1 = browser.find(e[0], "AllowableFieldType")
    aft2 = browser.find(e[1], "AllowableFieldType")
    print()
    print("{} {}".format(aft1.field_type.role, aft1))
    print("{} {}".format(aft2.field_type.role, aft2))
    
for edge in list(subgraph.edges)[:5]:
    print(subgraph[edge[0]][edge[1]])

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

## Guess sample compisition for sample properties

In [ ]:
frags = browser.search("level3")
example_sample = frags[0]

browser.set_verbose(False)

def collect_sample_dependencies(samples, edges=None):
    if edges is None:
        edges = []
    results = browser.recursive_retrieve(samples, {'field_values': 'sample'})
    retrieved_samples = results['sample']
    if not retrieved_samples:
        return edges
    else:
        edges += collect_sample_dependencies(retrieved_samples, edges=edges[:])
    for s in samples:
        for fv in s.field_values:
            if fv.sample:
                edge = (s.id, fv.sample.id)
                if edge not in edges:
                    edges.append(edge)
    return edges

def possible_items(samples, allowable_field_types):
    edges = collect_sample_dependencies(samples)
    sample_ids = reduce(lambda x, y: x + y, [list(x) for x in edges])
    samples = browser.find(sample_ids)
    for s in samples:
        print(s)
    object_type_ids = list(reduce((lambda x, y: set(x).union(set(y))), [[aft.object_type_id] for aft in all_afts]))
    potential_items = production.Item.where({"object_type_id": object_type_ids, "sample_id": sample_ids})
    return [item for item in potential_items if item.location != 'deleted']

In [ ]:
# sister edges

sister_edges = []

by_ot = {}
for aft in input_afts:
    by_ot.setdefault(aft.field_type.parent_id, []).append(aft)
    
for afts in by_ot.values():
    for aft1 in afts:
        for aft2 in afts:
            if not aft1.id != aft2.id:
                if  aft1.sample_type_id == aft1.sample_type_id or aft1.field_type.routing != aft2.field_type.routing:
                    sister_edges.append((aft1, aft2))
len(sister_edges)

In [ ]:
object_type_ids = list(reduce((lambda x, y: set(x).union(set(y))), [[aft.object_type_id] for aft in all_afts]))
sample_id = 24420
production.Item.where({"object_type_id": object_type_ids, "sample_id": sample_id})
# browser.where({"object_type_id": object_type_ids[-2:]}, "Item")

In [ ]:
from copy import deepcopy
from pydent.utils import filter_list
production.set_verbose(False)

graph = nx.DiGraph()
graph.add_nodes_from(subgraph.nodes(data=True))
graph.add_edges_from(subgraph.edges(data=True))

goal_object_type = "Yeast Glycerol Stock"
goal_sample = 'CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | PMOD6-PGRR-F10-yeGFP _X_ CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 | pMOD8-pGALZ4-URGR-F10 (JV)'

obj1 = production.ObjectType.find_by_name("Fragment Stock")
obj2 = production.ObjectType.find_by_name("Yeast Glycerol Stock")

afts1 = filter_list(input_afts, object_type_id=obj1.id, sample_type_id=obj1.sample_type_id)
afts2 = filter_list(output_afts, object_type_id=obj2.id, sample_type_id=obj2.sample_type_id)

# add item nodes
# items = possible_items([browser.find_by_name(goal_sample)], input_afts + output_afts)
# print("{} available items found".format(len(items)))

graph.add_node("START")

# item_node = lambda item: "Item{}".format(item.id)
# item_by_obj_id = {}
# for item in items:
#     item_by_obj_id.setdefault(item.object_type_id, []).append(item)

# for aft in all_afts:
#     for item in item_by_obj_id.get(aft.object_type_id, []):
#         graph.add_edge(item_node(item), aft.id, weight=0)

# for item in items:
#     graph.add_node(item_node(item), item=item)
#     graph.add_edge("START", item_node(item), weight=0)
for aft in input_afts:
    graph.add_edge("START", aft.id)


graph.add_node("END")
for aft in afts2:
    graph.add_edge(aft.id, "END", weight=0)

start_nodes = [aft.id for aft in afts1 if aft.id in nodes]
end_nodes = [aft.id for aft in afts2 if aft.id in nodes]

print("Input afts:")
for aftid in start_nodes:
    aft = browser.find(aftid, 'AllowableFieldType')
    try:
        print(aft.field_type.operation_type)
    except:
        pass
    

allpaths = []
print(list(graph.successors("START")))
for n1 in tqdm(list(graph.successors("START"))):
    for n2 in ["END"]:
        try:
            path = nx.dijkstra_path(graph, n1, n2, weight='weight')
            path_length = nx.dijkstra_path_length(graph, n1, n2, weight='weight')
            allpaths.append((path, path_length))
        except nx.exception.NetworkXNoPath:
            pass
            
allpaths = sorted(allpaths, key=lambda x: x[1])

print()
print("*"*50)
print("{} >> {}".format(obj1.name, obj2.name))
print("*"*50)
print()
for path, pathlen in allpaths[:10]:
    print(path)
    ots = []
    for aftid in path:
        aft = browser.find(aftid, 'AllowableFieldType')
        if aft:
            ot = browser.find(aft.field_type.parent_id, 'OperationType')
            ots.append("{ot}".format(role=aft.field_type.role, name=aft.field_type.name, ot=ot.name))
        edge_weights = [graph[x][y]['weight'] for x, y in zip(path[:-1], path[1:])]
    print(pathlen)
    print(edge_weights)
    print(len(ots))
    print(ots)
    print()


In [107]:
nx.floyd_warshall_numpy(graph, weight='weight')

matrix([[   0.        ,   15.19976896,   30.21387018, ...,           inf,
                   inf,           inf],
        [ 122.83990499,    0.        ,   15.01410122, ...,           inf,
                   inf,           inf],
        [ 107.82580377,  123.02557273,    0.        , ...,           inf,
                   inf,           inf],
        ..., 
        [          inf,           inf,           inf, ...,    0.        ,
                   inf,           inf],
        [          inf,           inf,           inf, ...,           inf,
            0.        ,           inf],
        [          inf,           inf,           inf, ...,           inf,
                   inf,    0.        ]])